<a href="https://colab.research.google.com/github/maytlim/doh_data/blob/main/vacancies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -s -O https://raw.githubusercontent.com/maytlim/doh_data/main/landline.py

In [2]:
import requests
import pandas as pd
import datetime
import landline
import gdown

In [3]:
  pd.options.display.float_format = '{:,.0f}'.format

In [4]:
try:
  import pdfx
except ModuleNotFoundError:
  !pip -q install pdfx
  import pdfx

     |████████████████████████████████| 81kB 3.2MB/s 
     |████████████████████████████████| 122kB 6.1MB/s 


In [5]:
#Code from https://gist.github.com/korakot/51a917e1f53891d53be223439b0f75c1
from google.colab import auth
auth.authenticate_user()  # must authenticate

'''list all ids of files directly under folder folder_id'''
def folder_list(folder_id):
  from googleapiclient.discovery import build
  gdrive = build('drive', 'v3', cache_discovery=False).files()
  #gdrive = build('drive', 'v3').files()
  res = gdrive.list(q="'%s' in parents" % folder_id).execute()
  return [f['id'] for f in res['files']]

'''download all files from a gdrive folder to current directory'''
def folder_download(folder_id):
  for fid in folder_list(folder_id):
    !gdown -q --id $fid

In [6]:
ddrop_readme = requests.get('http://bit.ly/DataDropPH').url
ddrop_readme = ddrop_readme.split('/')[-1].split('?')[0]
#print(ddrop_readme)
folder_download(ddrop_readme)

In [7]:
readme_pdf = !ls *.pdf
readme_pdf = readme_pdf[0][1:-1]
readme_pdf
pdfx_tmp = pdfx.PDFx(readme_pdf)
ddlinks = pdfx_tmp.get_references_as_dict()

In [8]:
month_day = readme_pdf.split('_')
month = int(month_day[0][-2:])
day = int(month_day[1][:2])
month, day
year = 2021
print('DOH DataDrop Release:', year, month, day)

DOH DataDrop Release: 2021 4 6


In [9]:
downloaded = False
for ddlink in ddlinks['url']:
  if downloaded == False:
    if ddlink[:15] == 'https://bit.ly/' and ddlink[-3:] != 'ive':
      datadrop = requests.get(ddlink).url
      datadrop = datadrop.split('/')[-1].split('?')[0]
      folder_download(datadrop)
      print('Downloading DOH Data Drop...')
      downloaded = True
  else:
    continue

In [10]:
data_date = datetime.datetime(year, month, day)
#data_dir ='DOH COVID Data Drop_'  + data_date.strftime('%Y%m%d') + '/'

In [11]:
landline_masterlist = landline.make_masterlist()

In [12]:
# hospital daily patient census
cap_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 05 DOH Data Collect - Daily Report.csv'
# hospital weekly inventory of supplies
sup_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 06 DOH Data Collect - Weekly Report.csv'
print('weekly supplies inventory:', sup_filename)
print('daily capability count:', cap_filename)
hosp_code = 'hfhudcode' 
#hosp_name = 'cfname'

weekly supplies inventory: DOH COVID Data Drop_ 20210406 - 06 DOH Data Collect - Weekly Report.csv
daily capability count: DOH COVID Data Drop_ 20210406 - 05 DOH Data Collect - Daily Report.csv


In [13]:
#load capacity data
df_cap = pd.read_csv(cap_filename)
df_cap['cap_reportdate'] = pd.to_datetime(df_cap['reportdate'])
df_cap.drop(['updateddate', 'addeddate', 'reportdate', 'id', 'north_coord', 'east_coord'], axis=1, inplace=True)
df_cap[hosp_code] = df_cap[hosp_code].apply(lambda x: x.strip())

In [14]:
!gdown -q 'https://raw.githubusercontent.com/maytlim/doh_data/main/nhfr.json'

In [15]:
df_nhfr = pd.read_json('nhfr.json')
print('Refer to https://nhfr.doh.gov.ph/ for the latest data')

Refer to https://nhfr.doh.gov.ph/ for the latest data


In [16]:
df_cap = df_cap.merge(how='left', right=df_nhfr, left_on='hfhudcode', right_on='Health Facility Code')

In [17]:
#load supplies data
#df_sup = pd.read_csv(sup_filename)
#df_sup['sup_reportdate'] = pd.to_datetime(df_sup['reportdate'])
#df_sup.drop(['updateddate', 'addeddate', 'reportdate', 'id'], axis=1, inplace=True)
#df_sup[hosp_code] = df_sup[hosp_code].apply(lambda x: x.strip())

In [18]:
# quarantine facility daily patient census
quarantine_daily = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 08 Quarantine Facility Data - Daily Report.csv')
# quarantine facility weekly inventory of supplies
#quarantine_weekly = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 09 Quarantine Facility Data - Weekly Report.csv')

In [19]:
# HCW availability
#baseline_v3 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 10 DOH Data Collect v3 - Baseline.csv')
# mechvent, swabbing, ipc
#baseline_v4 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 11 DOH Data Collect v4 - Baseline.csv')
# quarantine and isolation facilities
#ttmf = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 12 DDC TTMF Tracker v1 - Baseline.csv')

In [20]:
#print('{0}\n{1}'.format(df_cap.keys(), df_sup.keys()))
#print('{0}\n{1}'.format(quarantine_weekly.keys(), quarantine_daily.keys()))

In [21]:
reg_lookup = {'NCR': 'NATIONAL CAPITAL REGION (NCR)', 'ARMM': 'AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM)', 
              'CAR': 'CORDILLERA ADMINISTRA TIVE REGION (CAR)', 'I': 'REGION I (ILOCOS REGION)', 
              'II': 'REGION II (CAGAYAN VALLEY)', 'III': 'REGION III (CENTRAL LUZON)', 
              'IV-A': 'REGION IV-A (CALABAR ZON)', 'IV-B': 'REGION IV-B (MIMAROPA)', 
              'V': 'REGION V (BICOL REGION)', 'VI': 'REGION VI (WESTERN VISAYAS)', 
              'VII': 'REGION VII (CENTRAL VISAYAS)', 'VIII': 'REGION VIII (EASTERN VISAYAS)', 
              'IX': 'REGION IX (ZAMBOANGA PENINSULA)', 'X': 'REGION X (NORTHERN MINDANAO)', 
              'XI': 'REGION XI (DAVAO REGION)', 'XII': 'REGION XII (SOCCSKSA RGEN)', 
              'XIII': 'REGION XIII (CARAGA)'}

In [22]:
prefix = ['ics', 'resdoc', 'nurse', 'medtech', 'respthe', 'radtech', 'support']
suffix = ['_total', '_er', '_icu', '_ward'] 
triage = ['resdoc_triage', 'nurse_triage', 'support_triage']
added = ['Augmen_total']

In [23]:
def day_ago_from(n, from_date):
  showdate = from_date - datetime.timedelta(hours=24)*n
  return showdate.strftime('%Y-%m-%d')

In [24]:
def show_filtered(df_cap, days_ago, region='NATIONAL CAPITAL REGION (NCR)'):
  filter = (df_cap.mechvent_v > 0) & \
    (df_cap.cap_reportdate == day_ago_from(days_ago, data_date)) & \
    (df_cap.region == region) & \
    (df_cap.icu_v > 0) & \
    (df_cap.icu_o > 0)
  df_vacant = df_cap[filter][['cap_reportdate', 'cfname', 'city_mun', 'icu_v', 'isolbed_v', 'mechvent_v', 'beds_ward_v', 
                            'icu_o', 'isolbed_o', 'mechvent_o', 'beds_ward_o', 
                            'Landline Number', 'Landline Number 2', 'Fax Number', 'Ownership Major Classification']]
  df_vacant.rename({'Landline Number': 'landline', 'Landline Number 2': 'landline2', 'Fax Number': 'fax', 'Ownership Major Classification':'ownership'}, axis=1, inplace=True)
  df_vacant['landline'] = df_vacant['landline'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['landline2'] = df_vacant['landline2'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['fax'] = df_vacant['fax'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant.sort_values(by='icu_v', ascending=False, inplace=True)
  df_vacant.reset_index(inplace=True)
  return df_vacant

In [25]:
show_filtered(df_cap, days_ago=3)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,46162,2021-04-03,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,64,103,41,0,36,297,19,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
1,235321,2021-04-03,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,20,17,9,0,10,19,6,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
2,287646,2021-04-03,JUSTICE JOSE ABAD SANTOS GENERAL HOSPITAL,BINONDO,8,33,5,0,4,21,1,2,(02) 8254 2453,(02) 5310 1984,None,Government
3,324476,2021-04-03,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,6,3,3,3,4,8,10,13,(02) 8716 1853,None,(02) 8716 1853,Private
4,56976,2021-04-03,ASIAN HOSPITAL INC.,CITY OF MUNTINLUPA,6,37,30,0,4,25,15,0,(02) 8771 9000,None,(02) 8876 5710,Private
5,3383,2021-04-03,"CAPITOL MEDICAL CENTER, INC.",QUEZON CITY,5,1,5,0,5,14,5,8,(02) 8372 3825,(02) 8938 7789,(02) 3411 4320,Private
6,217368,2021-04-03,NATIONAL CHILDREN?S HOSPITAL,QUEZON CITY,3,33,3,0,10,44,10,0,(02) 8724 0656,None,(02) 8721 9139,Government
7,41455,2021-04-03,ADVENTIST MEDICAL CENTER MANILA,PASAY CITY,3,4,4,0,1,25,2,0,(02) 8525 9191,None,(02) 8524 3256,Private
8,100087,2021-04-03,VALENZUELA MEDICAL CENTER,CITY OF VALENZUELA,3,1,6,22,4,1,2,31,(02) 8294 6711,None,(02) 8291 4259,Government
9,259427,2021-04-03,EAST AVENUE MEDICAL CENTER,QUEZON CITY,3,3,21,0,57,57,36,0,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government


In [26]:
show_filtered(df_cap, days_ago=2)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,46163,2021-04-04,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,61,105,38,0,39,295,22,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
1,235322,2021-04-04,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,15,18,9,0,2,36,6,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
2,287647,2021-04-04,JUSTICE JOSE ABAD SANTOS GENERAL HOSPITAL,BINONDO,8,35,5,0,4,19,1,2,(02) 8254 2453,(02) 5310 1984,None,Government
3,56977,2021-04-04,ASIAN HOSPITAL INC.,CITY OF MUNTINLUPA,6,37,30,0,4,25,15,0,(02) 8771 9000,None,(02) 8876 5710,Private
4,198294,2021-04-04,QUIRINO MEMORIAL MEDICAL CENTER,QUEZON CITY,5,22,12,0,5,33,8,74,(02) 8913 4758,(02) 8421 2250,(02) 8913 4758,Government
5,324477,2021-04-04,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,5,2,2,3,5,9,11,13,(02) 8716 1853,None,(02) 8716 1853,Private
6,100088,2021-04-04,VALENZUELA MEDICAL CENTER,CITY OF VALENZUELA,3,1,8,15,4,1,0,38,(02) 8294 6711,None,(02) 8291 4259,Government
7,6160,2021-04-04,DE LOS SANTOS MEDICAL CENTER,QUEZON CITY,3,0,3,13,5,18,4,6,(02) 8723 0041,(02) 8893 5762,None,Private
8,339442,2021-04-04,AMANG RODRIGUEZ MEMORIAL MEDICAL CENTER,CITY OF MARIKINA,2,0,1,5,3,57,2,40,(02) 8942 0245,None,None,Government
9,195805,2021-04-04,DR. FE DEL MUNDO MEDICAL CENTER,QUEZON CITY,2,0,2,1,4,0,3,37,(02) 8712 0845,None,(02) 8712 5796,Private


In [27]:
show_filtered(df_cap, days_ago=1)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,46164,2021-04-05,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,51,121,34,0,49,279,26,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
1,235323,2021-04-05,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,15,25,9,0,5,37,6,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
2,287648,2021-04-05,JUSTICE JOSE ABAD SANTOS GENERAL HOSPITAL,BINONDO,8,39,5,0,4,15,1,2,(02) 8254 2453,(02) 5310 1984,None,Government
3,198295,2021-04-05,QUIRINO MEMORIAL MEDICAL CENTER,QUEZON CITY,6,21,11,0,4,36,9,72,(02) 8913 4758,(02) 8421 2250,(02) 8913 4758,Government
4,324478,2021-04-05,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,4,5,2,2,6,6,11,14,(02) 8716 1853,None,(02) 8716 1853,Private
5,6161,2021-04-05,DE LOS SANTOS MEDICAL CENTER,QUEZON CITY,3,4,2,8,5,14,5,10,(02) 8723 0041,(02) 8893 5762,None,Private
6,259429,2021-04-05,EAST AVENUE MEDICAL CENTER,QUEZON CITY,3,39,22,0,57,167,44,0,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
7,100089,2021-04-05,VALENZUELA MEDICAL CENTER,CITY OF VALENZUELA,3,1,5,10,4,1,3,43,(02) 8294 6711,None,(02) 8291 4259,Government
8,267731,2021-04-05,TONDO MEDICAL CENTER,TONDO I / II,2,18,4,0,16,12,2,96,(02) 8522 1174,(02) 8522 9244,(02) 8252 8661,Government
9,195806,2021-04-05,DR. FE DEL MUNDO MEDICAL CENTER,QUEZON CITY,2,0,3,5,4,0,2,33,(02) 8712 0845,None,(02) 8712 5796,Private


In [28]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['II'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership


In [29]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['IV-A'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,206862,2021-04-05,THE MEDICAL CITY SOUTH LUZON,CITY OF SANTA ROSA,7,0,1,4,3,12,3,11,+63 495440120,None,+63 495440120-1000,Private
1,57256,2021-04-05,"MARY MEDIATRIX MEDICAL CENTER, INC.",LIPA CITY,4,1,7,0,7,0,0,32,+63 437736800,+63 437736861,+63 437736861,Private
2,83265,2021-04-05,GENERAL EMILIO AGUINALDO MEMORIAL HOSPITAL,TRECE MARTIRES CITY (CAPITAL),4,17,10,12,1,12,1,12,+63 464190063,None,None,Government
3,282663,2021-04-05,DE LA SALLE UNIVERSITY MEDICAL CENTER,CITY OF DASMARIÑAS,4,1,10,0,4,37,17,0,(02) 8988 3100,+63 464818000,None,Private
4,132450,2021-04-05,"ST. JAMES HOSPITAL, INC.",CITY OF SANTA ROSA,3,1,3,0,1,3,0,0,+63 495341254,None,None,Private
5,248340,2021-04-05,DANIEL O. MERCADO MEDICAL CENTER,CITY OF TANAUAN,3,3,2,0,1,1,0,11,+63 437781810,+63 437780960,None,Private
6,268272,2021-04-05,UNIHEALTH-SOUTHWOODS HOSPITAL AND MEDICAL CENT...,CITY OF BIÑAN,3,21,15,7,2,0,0,8,+63 498616385,None,None,Private
7,277408,2021-04-05,BATANGAS MEDICAL CENTER,BATANGAS CITY (CAPITAL),2,22,27,0,9,9,3,36,+63 437408307,None,+63 437230165,Government
8,289589,2021-04-05,MEDICAL CENTER WESTERN BATANGAS,BALAYAN,2,3,1,3,2,3,0,4,+63 434071103,None,+63 434071101,Private
9,313968,2021-04-05,"GLOBAL CARE MEDICAL CENTER OF CANLUBANG, INC.",CITY OF CALAMBA,2,1,2,8,3,0,1,16,+63 495205626,None,None,Private


In [30]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['III'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,240317,2021-04-05,JOSE B. LINGAD MEMORIAL REGIONAL HOSPITAL,CITY OF SAN FERNANDO (CAPITAL),16,0,20,4,66,0,8,241,+63 459613544,None,None,Government
1,4774,2021-04-05,BATAAN GENERAL HOSPITAL AND MEDICAL CENTER,CITY OF BALANGA (CAPITAL),11,66,20,15,28,33,28,21,+63 472372269,None,None,Government
2,305085,2021-04-05,BULACAN MEDICAL CENTER,CITY OF MALOLOS (CAPITAL),6,63,46,22,24,24,1,78,+63 447910630,None,None,Government
3,239209,2021-04-05,DR. PAULINO J. GARCIA MEMORIAL RESEARCH & MEDI...,CABANATUAN CITY,4,28,8,15,12,20,8,41,+63 444638286,+63 444639937,None,Government
4,42011,2021-04-05,"GRACE GENERAL HOSPITAL, INC.",CITY OF SAN JOSE DEL MONTE,3,1,1,0,1,13,0,0,+63 445194838,None,None,Private
5,218480,2021-04-05,SACRED HEART MEDICAL CENTER,ANGELES CITY,2,2,2,13,1,0,1,6,+63 456257876,None,None,Private
6,69694,2021-04-05,"CENTRAL LUZON DOCTORS? HOSPITAL, INC.",CITY OF TARLAC (CAPITAL),1,34,4,0,7,17,1,0,+63 459820806,None,None,Private
7,81893,2021-04-05,PRES. RAMON MAGSAYSAY MEMORIAL HOSPITAL,IBA (CAPITAL),1,9,3,25,7,28,1,155,+63 478117212,None,None,Government
8,208239,2021-04-05,"NUEVA ECIJA DOCTORS? HOSPITAL, INC.",CABANATUAN CITY,1,20,3,5,2,8,0,4,+63 449400176,None,None,Private
9,230333,2021-04-05,JAMES L. GORDON MEMORIAL HOSPITAL,OLONGAPO CITY,1,38,2,2,3,31,3,6,+63 476021229,+63 476021230,+63 476021228,Government


In [31]:
roi = reg_lookup['NCR']
#roi = reg_lookup['II']
#roi = reg_lookup['IV-A']
filter = (df_cap.cap_reportdate == day_ago_from(1, data_date)) & (df_cap.region == roi)
occupied = df_cap[filter]['icu_o'].sum()
vacant = df_cap[filter]['icu_v'].sum()
pct_used = (occupied / (vacant + occupied)) * 100